[View in Colaboratory](https://colab.research.google.com/github/JozeeLin/google-tensorflow-exercise/blob/master/%E7%BB%8F%E5%85%B8%E5%8D%B7%E7%A7%AF%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C_GoogleInceptionNet.ipynb)

## Google Inception Net
Inception V1的最大特点是控制了计算量和参数量的同时，获得了非常好的分类性能。

**Inception V1**降低参数量的目的有两点：
1. 参数越多模型越庞大，需要供模型学习的数量就越大，而目前高质量的数据非常昂贵
2. 参数越多，耗费的计算资源也会更大

Inception V1参数少但效果好的原因除了模型层数更深、表达能力更强外，还有两点：

- 一是去除了最后的全连接层，用全局平均池化层(即将图片尺寸变为1x1)来取代它。
  全连接层几乎占据了AlexNet和VGGNet中90%的参数量，而且会引起过拟合，去除全连接层后模型训练更快并且减轻了过拟合。
  用全局平均池化层去打全连接层的做法借鉴了Network In Network(NIN)论文。
  
- 二是Inception V1中精心设计的Inception Module提高了参数的利用效率。这一部分也借鉴了NIN思想。形象的解释就是Inception Module本身如同大网络中的一个小网络，其结构可以反复堆叠在一起形成大网络。

MLPConv基本等效于普通卷积层后在连接1x1的卷积核ReLU激活函数。

Inception Module的基本结构，其中有4个分支：
- 第一个分支对输入进行1x1的卷积，这其实也是NIN种提出的一个重要结构。1x1卷积是一个非常优秀的结构，它可以跨通道组织信息，提高网络的表达能力，同时可以对输出通道降维和升维。
- 第二个分支先使用了1x1卷积，然后连接3x3卷积，相当于进行了两次特征变换。
- 第三个分支类似，先是1x1的卷积，然后连接5x5卷积
- 最后一个分支，则是3x3最大池化后直接使用1x1卷积

Inception Module的4个分支在最后通过一个聚合操作合并。此算法一共包含了3种不同尺寸的卷积和一个最大池化，增加了网络对不同尺度的适应性，这一部分和Multi-Scale的思想类似。

Inception Module的主要目标是找到最优的稀疏结构单元，论文中提到其稀疏结构基于Hebbian原理。

Hebbian原理：神经反射活动的持续与重复会导致神经元连接稳定性的持久提升， 当两个神经元细胞A和B距离很近，并且A参与了对B重复、持续的兴奋，那么某些代谢变化会导致A将作为能使B兴奋的细胞。学习过程中的刺激会使神经元间的突触强度增加。

> 延伸阅读:s受Hebbian原理的影响，另一篇文章Provable Bounds for Learning Some Deep Representations提出，如果数据集的概率分布可以被一个很大很稀疏的神经网络所表达，那么构筑这个网络的最佳方法是逐层构筑网络：将上一层高度相关的节点聚类，并将聚类出来的每一个小簇连接到一起。

**Inception V2**学习了VGGNet，用两个3x3的卷积代替5x5的大卷积(用以降低参数量并减轻过拟合)，还提出了著名的Batch Normalization(简称BN)方法。

BN是一个非常有效的正则化方法，可以让大型卷积网络的训练速度加快很多倍，同时收敛后的分类准确率也可以得到大幅提高。

BN在用于神经网络某层时，会对每一个mini-batch数据的内部进行标准化处理，使输出规范化到N(0,1)的正态分布，减少了Internal Covariate Shift(内部神经元分布的改变)。

BN的论文提出，传统的深度神经网络在训练师，每一层的输入的分布都在变化，导致训练变得困难，我们只能使用一个很小的学习速率解决这个问题。

而对每一层使用BN之后，我们就可以有效地解决这个问题，学习速率可以增大很多倍，达到之前的准确率所需要的迭代次数只有1/14，训练时间大大缩短。

而达到之后的准确率后，可以继续训练，并最终取得远超于Inception V1模型的性能。

**Inception V3**主要有两方面的改造：
- 一是引入了Factorization into small convolutions的思想，将一个较大的二维卷积拆成两个较小的一维卷积，比如将7x7卷积拆成1x7卷积和7x1卷积，
  一方面节约了大量参数，加速运算并减轻了过拟合，同时增加了一层非线性扩展模型表达能力。论文中指出，这种非对称的卷积结构拆分，
  其结果比对称地拆为几个相同的小卷积核效果更明显，可以处理更多、更丰富的空间特征，增加特征多样性。
  
- 另一方面，优化了Inception Module的结构，即使用了Network In Network In Network
  
**Inception V4**相比V3主要是结合了微软的ResNet。

### 本节要实现的是V3.

In [0]:
import tensorflow as tf
import time
from datetime import datetime
import math
slim = tf.contrib.slim
#产生截断的正态分布
trunc_normal = lambda stddev: tf.truncated_normal_initializer(0.0, stddev)

In [0]:
#用来生成网络中经常用到的函数的默认参数，比如卷积网络的激活函数、权重初始化方式、标准化等

def inception_v3_arg_scope(weight_decay=0.00004, stddev=0.1, batch_norm_var_collection='moving_vars'):
  '''
  weight_decay:L2正则
  stddev:标准差
  '''
  batch_norm_params = { #参数字典
      'decay': 0.9997, #衰减系数
      'epsilon': 0.001,
      'updates_collections':tf.GraphKeys.UPDATE_OPS,
      'variables_collections':{
          'beta': None,
          'gamma':None,
          'moving_mean':[batch_norm_var_collection],
          'moving_variance':[batch_norm_var_collection],
      }
  }
  
  with slim.arg_scope([slim.conv2d, slim.fully_connected],
                     weights_regularizer=slim.l2_regularizer(weight_decay)):
    with slim.arg_scope(
        [slim.conv2d],
        weights_initializer=tf.truncated_normal_initializer(stddev=stddev),
        activation_fn=tf.nn.relu,
        normalizer_fn = slim.batch_norm,
        normalizer_params=batch_norm_params
    ) as sc:
      return sc

In [0]:
#用于生成Inception V3网络的卷积部分，参数inputs为输入的图片数据的tensor，scope为包含了函数默认参数的环境。
def inception_v3_base(inputs, scope=None):
  end_points = {}#用于保存某些关键节点供之后使用
  
  with tf.variable_scope(scope, 'InceptionV3', [inputs]):
    #对slim.conv2d、slim.max_pool2d、slim_avg_pool2d这三个函数设置默认参数值，stride设为1，padding设为VALID
    with slim.arg_scope([slim.conv2d, slim.max_pool2d, slim.avg_pool2d], stride=1, padding='VALID'):
      #非Inception Module的卷积层，第一个参数为输入tensor，参数2为输出的通道数，参数3为卷积核尺寸，4为步长，5为padding的模式默认为VALID
      net = slim.conv2d(inputs, 32, [3,3], stride=2, scope='Conv2d_1a_3x3')
      net = slim.conv2d(net, 32, [3,3], scope='Conv2_2a_3x3')
      net = slim.conv2d(net, 64, [3,3], padding='SAME', scope='Conv2d_2b_3x3')
      
      net = slim.max_pool2d(net, [3,3], stride=2, scope='MaxPool_3a_3x3')#重叠最大池化层
      
      net = slim.conv2d(net, 80, [1,1], scope='Conv2d_3b_1x1')
      net = slim.conv2d(net, 192, [3,3], scope='Conv2d_4a_3x3')
      
      net = slim.max_pool2d(net,[3,3], stride=2,scope='MaxPool_5a_3x3')
      
      
  #接下来将是三个连续的Inception模块组，这三个Inception模块组中各自分别有多个Inception Module，这部分的网络结构即是Inception V3的精华所在。
      
  #第一个Inception模块组包含了3个结构类似的Inception Module
  with slim.arg_scope([slim.conv2d,slim.max_pool2d, slim.avg_pool2d], stride=1, padding='SAME'):
    with tf.variable_scope('Mixed_5b'):
      with tf.variable_scope('Branch_0'):
        branch_0 = slim.conv2d(net, 64, [1,1], scope='Conv2d_0a_1x1')
        
      with tf.variable_scope('Branch_1'):
        branch_1 = slim.conv2d(net, 48, [1,1], scope='Conv2d_0a_1x1')
        branch_1 = slim.conv2d(branch_1, 64, [5,5], scope='Conv2d_0b_5x5')
        
      with tf.variable_scope('Branch_2'):
        branch_2 = slim.conv2d(net, 64, [1,1], scope='Conv2d_0a_1x1')
        branch_2 = slim.conv2d(branch_2, 96, [3,3], scope='Conv2d_0b_3x3')
        branch_2 = slim.conv2d(branch_2, 96, [3,3], scope='Conv2d_0c_3x3')
        
      with tf.variable_scope('Branch_3'):
        branch_3 = slim.avg_pool2d(net,[3,3], scope='AvgPool_0a_3x3')
        branch_3 = slim.conv2d(branch_3, 32, [1,1], scope='Conv2d_0b_1x1')
        
      net = tf.concat([branch_0, branch_1, branch_2, branch_3], 3)
      
    with tf.variable_scope('Mixed_5c'):
      with tf.variable_scope('Branch_0'):
        branch_0 = slim.conv2d(net, 64, [1,1], scope='Conv2d_0a_1x1')
      with tf.variable_scope('Branch_1'):
        branch_1 = slim.conv2d(net, 48, [1,1], scope='Conv2d_0b_1x1')
        branch_1 = slim.conv2d(branch_1, 64, [5,5], scope='Conv_1_0c_5x5')
        
      with tf.variable_scope('Branch_2'):
        branch_2 = slim.conv2d(net, 64, [1,1], scope='Conv2_0a_1x1')
        branch_2 = slim.conv2d(branch_2, 96, [3,3], scope='Conv2d_0b_3x3')
        branch_2 = slim.conv2d(branch_2, 96, [3,3], scope='Conv2d_0c_3x3')
        
      with tf.variable_scope('Branch_3'):
        branch_3 = slim.avg_pool2d(net, [3,3], scope='AvgPool_0a_3x3')
        branch_3 = slim.conv2d(branch_3, 64, [1,1], scope='Conv2_0b_1x1')
        
      net = tf.concat([branch_0,branch_1, branch_2,branch_3],3)
      
      
    #第一个模块组的第三个Inception Module，4个分支结构
    with tf.variable_scope('Mixed_5d'):
      with tf.variable_scope('Branch_0'):
        branch_0 = slim.conv2d(net, 64, [1,1], scope='Conv2d_0a_1x1')
        
      with tf.variable_scope('Branch_1'):
        branch_1 = slim.conv2d(net, 48, [1,1], scope='Conv2d_0a_1x1')
        branch_1 = slim.conv2d(branch_1, 64, [5,5], scope='Conv2d_0b_5x5')
        
      with tf.variable_scope('Branch_2'):
        branch_2 = slim.conv2d(net, 64, [1,1], scope='Conv2d_0a_1x1')
        branch_2 = slim.conv2d(branch_2, 96, [3,3], scope='Conv2d_0b_3x3')
        branch_2 = slim.conv2d(branch_2, 96, [3,3], scope='Conv2d_0c_3x3')
        
      with tf.variable_scope('Branch_3'):
        branch_3 = slim.avg_pool2d(net, [3,3], scope='AvgPool_0a_3x3')
        branch_3 = slim.conv2d(branch_3, 64, [1,1], scope='Conv2d_0b_1x1')
        
      net = tf.concat([branch_0, branch_1, branch_2, branch_3], 3)
      
    #第二个模块组，包含5个Inception Module
    
    #第一个Inception Module，含有3个分支
    with tf.variable_scope('Mixed_6a'):
      #第一个分支是一个384输出通道的3x3卷积
      with tf.variable_scope('Branch_0'):
        branch_0 = slim.conv2d(net, 384,[3,3], stride=2, padding='VALID', scope='Conv2d_1a_1x1')
      #第二个分支有三层，分别是一个64输出通道的1x1卷积和两个96输出通道的3x3卷积
      with tf.variable_scope('Branch_1'):
        branch_1 = slim.conv2d(net, 64, [1,1], scope='Conv2d_0a_1x1')
        branch_1 = slim.conv2d(branch_1, 96, [3,3], scope='Conv2d_0b_3x3')
        branch_1 = slim.conv2d(branch_1, 96, [3,3], stride=2, padding='VALID', scope='Conv2d_1a_1x1')
      #第三个分支是一个3x3最大池化层
      with tf.variable_scope('Branch_2'):
        branch_2 = slim.max_pool2d(net,[3,3], stride=2, padding='VALID', scope='MaxPool_1a_3x3')
      #将三个分支在输出通道上合并
      net = tf.concat([branch_0, branch_1, branch_2], 3)
      
    #第二个Inception Module，含有4个分支
    with tf.variable_scope('Mixed_6b'):
      with tf.variable_scope('Branch_0'):
        branch_0 = slim.conv2d(net, 192, [1,1], scope='Conv2d_0a_1x1')
       
      with tf.variable_scope('Branch_1'):
        branch_1 = slim.conv2d(net, 128, [1,1], scope='Conv2d_0a_1x1')
        branch_1 = slim.conv2d(branch_1, 128, [1,7], scope='Conv2d_0b_1x7')
        branch_1 = slim.conv2d(branch_1, 192, [7,1], scope='Conv2d_0c_7x1')
        
      with tf.variable_scope('Branch_2'):
        branch_2 = slim.conv2d(net, 128, [1,1], scope='Conv2d_0a_1x1')
        branch_2 = slim.conv2d(branch_2, 128, [7,1], scope='Conv2d_0b_7x1')
        branch_2 = slim.conv2d(branch_2, 128, [1,7], scope='Conv2d_0c_1x7')
        branch_2 = slim.conv2d(branch_2, 128, [7,1], scope='Conv2d_0d_7x1')
        branch_2 = slim.conv2d(branch_2, 192, [1,7], scope='Conv2d_0e_1x7')
        
      with tf.variable_scope('Branch_3'):
        branch_3 = slim.avg_pool2d(net, [3,3], scope='AvgPool_0a_3x3')
        branch_3 = slim.conv2d(branch_3, 192, [1,1], scope='Conv2d_0b_1x1')
        
      net = tf.concat([branch_0, branch_1, branch_2, branch_3], 3)
      
    #第三个Inception Module，含有4个分支
    with tf.variable_scope('Mixed_6c'):
      with tf.variable_scope('Branch_0'):
        branch_0 = slim.conv2d(net, 192,[1,1], scope='Conv2d_0a_1x1')
        
      with tf.variable_scope('Branch_1'):
        branch_1 = slim.conv2d(net, 160, [1,1], scope='Conv2d_0a_1x1')
        branch_1 = slim.conv2d(branch_1, 160, [1,7], scope='Conv2d_0b_1x7')
        branch_1 = slim.conv2d(branch_1, 192, [7,1], scope='Conv2d_0c_7x1')
        
      with tf.variable_scope('Branch_2'):
        branch_2 = slim.conv2d(net, 160, [1,1], scope='Conv2d_0a_1x1')
        branch_2 = slim.conv2d(branch_2, 160, [7,1], scope='Conv2d_0b_7x1')
        branch_2 = slim.conv2d(branch_2, 160, [1,7], scope='Conv2d_0c_1x7')
        branch_2 = slim.conv2d(branch_2, 160, [7,1], scope='Conv2d_0d_7x1')
        branch_2 = slim.conv2d(branch_2, 160, [1,7], scope='Conv2d_0e_1x7')
        
      with tf.variable_scope('Branch_3'):
        branch_3 = slim.avg_pool2d(net, [3,3], scope='AvgPool_0a_3x3')
        branch_3 = slim.conv2d(branch_3, 192, [1,1], scope='Conv2d_0b_1x1')
        
      net = tf.concat([branch_0, branch_1, branch_2, branch_3], 3)
      
    #第四个Inception Module
    with tf.variable_scope('Mixed_6d'):
      with tf.variable_scope('Branch_0'):
        branch_0 = slim.conv2d(net, 192, [1,1], scope='Conv2d_0a_1x1')
        
      with tf.variable_scope('Branch_1'):
        branch_1 = slim.conv2d(net, 160, [1,1], scope='Conv2d_0a_1x1')
        branch_1 = slim.conv2d(branch_1, 160, [1,7], scope='Conv2d_0b_1x7')
        branch_1 = slim.conv2d(branch_1, 192, [7,1], scope='Conv2d_0c_7x1')
        
      with tf.variable_scope('Branch_2'):
        branch_2 = slim.conv2d(net, 160, [1,1], scope='Conv2d_0a_1x1')
        branch_2 = slim.conv2d(branch_2, 160, [7,1], scope='Conv2d_0b_7x1')
        branch_2 = slim.conv2d(branch_2, 160, [1,7], scope='Conv2d_0c_1x7')
        branch_2 = slim.conv2d(branch_2, 160, [7,1], scope='Conv2d_0d_7x1')
        branch_2 = slim.conv2d(branch_2, 160, [1,7], scope='Conv2d_0e_1x7')
        
      with tf.variable_scope('Branch_3'):
        branch_3 = slim.avg_pool2d(net, [3,3], scope='AvgPool_0a_3x3')
        branch_3 = slim.conv2d(branch_3, 192, [1,1], scope='Conv2d_0b_1x1')
        
      net = tf.concat([branch_0,branch_1,branch_2,branch_3], 3)
      
    #第五个Inception Module
    with tf.variable_scope('Mixed_6e'):
      with tf.variable_scope('Branch_0'):
        branch_0 = slim.conv2d(net, 192, [1,1], scope='Conv2d_0a_1x1')
        
      with tf.variable_scope('Branch_1'):
        branch_1 = slim.conv2d(net, 192, [1,1], scope='Conv2d_0a_1x1')
        branch_1 = slim.conv2d(branch_1,192, [1,7], scope='Conv2d_0b_1x7')
        branch_1 = slim.conv2d(branch_1,192, [7,1], scope='Conv2d_0c_7x1')
        
      with tf.variable_scope('Branch_2'):
        branch_2 = slim.conv2d(net, 192, [1,1], scope='Conv2d_0a_1x1')
        branch_2 = slim.conv2d(branch_2, 192, [7,1], scope='Conv2d_0b_7x1')
        branch_2 = slim.conv2d(branch_2, 192, [1,7], scope='Conv2d_0c_1x7')
        branch_2 = slim.conv2d(branch_2, 192, [7,1], scope='Conv2d_0d_7x1')
        branch_2 = slim.conv2d(branch_2, 192, [1,7], scope='Conv2d_0e_1x7')
        
      with tf.variable_scope('Branch_3'):
        branch_3 = slim.avg_pool2d(net, [3,3], scope='AvgPool_0a_3x3')
        branch_3 = slim.conv2d(branch_3, 192, [1,1], scope='Conv2d_0b_1x1')
        
      net = tf.concat([branch_0, branch_1, branch_2, branch_3],3)
      
    end_points['Mixed_6e'] = net
    
    #第三个模块组，包含3个Inception Module
    
    #第一个Inception Module，包含3个分支
    with tf.variable_scope('Mixed_7a'):
      with tf.variable_scope('Branch_0'):
        branch_0 = slim.conv2d(net, 192, [1,1], scope='Conv2d_0a_1x1')
        branch_0 = slim.conv2d(branch_0, 320, [3,3], stride=2, padding='VALID', scope='Conv2d_1a_3x3')
      
      with tf.variable_scope('Branch_1'):
        branch_1 = slim.conv2d(net, 192,[1,1], scope='Conv2d_0a_1x1')
        branch_1 = slim.conv2d(branch_1, 192,[1,7],scope='Conv2d_0b_1x7')
        branch_1 = slim.conv2d(branch_1, 192,[7,1], scope='Conv2d_0c_7x1')
        branch_1 = slim.conv2d(branch_1, 192,[3,3], stride=2, padding='VALID',scope='Conv2d_1a_3x3')
        
      with tf.variable_scope('Branch_2'):
        branch_2 = slim.max_pool2d(net, [3,3], stride=2, padding='VALID', scope='MaxPool_1a_3x3')
        
      net = tf.concat([branch_0, branch_1, branch_2], 3)
      
    #第二个Inception Module，包含4个分支
    with tf.variable_scope('Mixed_7b'):
      with tf.variable_scope('Branch_0'):
        branch_0 = slim.conv2d(net, 320, [1,1], scope='Conv2d_0a_1x1')
        
      with tf.variable_scope('Branch_1'):
        branch_1 = slim.conv2d(net, 384, [1,1], scope='Conv2d_0a_1x1')
        branch_1 = tf.concat([
            slim.conv2d(branch_1, 384, [1,3], scope='Conv2d_0b_1x3'),
            slim.conv2d(branch_1, 384, [3,1], scope='Conv2d_0b_3x1')
        ],3)
        
      with tf.variable_scope('Branch_2'):
        branch_2 = slim.conv2d(net, 448, [1,1], scope='Conv2d_0a_1x1')
        branch_2 = slim.conv2d(branch_2, 384, [3,3], scope='Conv2d_0b_3x3')
        branch_2 = tf.concat([
            slim.conv2d(branch_2, 384, [1,3], scope='Conv2d_0c_1x3'),
            slim.conv2d(branch_2, 384, [3,1], scope='Conv2d_0d_3x1')
        ],3)
        
      with tf.variable_scope('Branch_3'):
        branch_3 = slim.avg_pool2d(net, [3,3], scope='AvgPool_0a_3x3')
        branch_3 = slim.conv2d(branch_3, 192, [1,1], scope='Conv2d_0b_1x1')
        
      net = tf.concat([branch_0, branch_1, branch_2, branch_3], 3)
      
    #第三个Inception Module，包含4分分支
    with tf.variable_scope('Mixed_7c'):
      with tf.variable_scope('Branch_0'):
        branch_0 = slim.conv2d(net, 320, [1,1], scope='Conv2d_0a_1x1')
        
      with tf.variable_scope('Branch_1'):
        branch_1 = slim.conv2d(net, 384, [1,1], scope='Conv2d_0a_1x1')
        branch_1 = tf.concat([
            slim.conv2d(branch_1, 384, [1,3], scope='Conv2d_0b_1x3'),
            slim.conv2d(branch_1, 384, [3,1], scope='Conv2d_0c_3x1')
        ],3)
        
      with tf.variable_scope('Branch_2'):
        branch_2 = slim.conv2d(net, 448, [1,1], scope='Conv2d_0a_1x1')
        branch_2 = slim.conv2d(branch_2, 384, [3,3], scope='Conv2d_0b_3x3')
        branch_2 = tf.concat([
            slim.conv2d(branch_2, 384,[1,3], scope='Conv2d_0c_1x3'),
            slim.conv2d(branch_2, 384, [3,1],scope='Conv2d_0d_3x1')
        ],3)
        
      with tf.variable_scope('Branch_3'):
        branch_3 = slim.avg_pool2d(net, [3,3], scope='AvgPool_0a_3x3')
        branch_3 = slim.conv2d(branch_3, 192, [1,1], scope='Conv2d_0b_1x1')
        
      net = tf.concat([branch_0, branch_1, branch_2, branch_3], 3)
  return net, end_points

In [0]:
def inception_v3(inputs, 
                num_classes=1000,
                is_training=True,
                dropout_keep_prob=0.8,
                prediction_fn=slim.softmax,
                spatial_squeeze=True,
                reuse=None,
                scope='InceptionV3'):
  with tf.variable_scope(scope, 'InceptionV3', [inputs, num_classes],reuse=reuse) as scope:
    with slim.arg_scope([slim.batch_norm, slim.dropout],
                       is_training=is_training):
      net, end_points = inception_v3_base(inputs, scope=scope)
      
    with slim.arg_scope([slim.conv2d, slim.max_pool2d, slim.avg_pool2d], stride=1, padding='SAME'):
      aux_logits = end_points['Mixed_6e']
      with tf.variable_scope('AuxLogits'):
        aux_logits = slim.avg_pool2d(
          aux_logits,[5,5],stride=3, padding='VALID',scope='AvgPool_1a_5x5'
        )
        aux_logits = slim.conv2d(aux_logits, 128, [1,1], scope='Conv2d_1b_1x1')
        
        aux_logits = slim.conv2d(
          aux_logits, 768, [5,5],
            weights_initializer=trunc_normal(0.01),
            padding='VALID', scope='Conv2d_2a_5x5'
        )
        aux_logits = slim.conv2d(
                    aux_logits, num_classes, [1, 1], activation_fn=None,
                    normalizer_fn=None, weights_initializer=trunc_normal(0.001),
                    scope='Conv2d_2b_5x5')
        if spatial_squeeze:
          aux_logits = tf.squeeze(aux_logits, [1, 2], name='SpatialSqueeze')
          end_points['AuxLogits'] = aux_logits
          
      with tf.variable_scope('Logits'):
        net = slim.avg_pool2d(net, [8, 8], padding='VALID',
                              scope='AvgPool_1a_8x8')
        net = slim.dropout(net, keep_prob=dropout_keep_prob,
                           scope='Dropout_1b')
        end_points['PreLogits'] = net
        logits = slim.conv2d(net, num_classes, [1, 1], activation_fn=None,
                             normalizer_fn=None, scope='Conv2d_1c_1x1')
        if spatial_squeeze:
            logits = tf.squeeze(logits, [1, 2], name='SpatialSqueeze')
            end_points['logits'] = logits
            end_points['Predictions'] = prediction_fn(logits, scope='Predictions')
        return logits, end_points
      
    with tf.variable_scope('Logits'):
      net = slim.avg_pool2d(net, [8, 8], padding='VALID',
                            scope='AvgPool_1a_8x8')
      net = slim.dropout(net, keep_prob=dropout_keep_prob,
                           scope='Dropout_1b')
      end_points['PreLogits'] = net
      logits = slim.conv2d(net, num_classes, [1, 1], activation_fn=None,
                             normalizer_fn=None, scope='Conv2d_1c_1x1')
      if spatial_squeeze:
          logits = tf.squeeze(logits, [1, 2], name='SpatialSqueeze')
          end_points['logits'] = logits
          end_points['Predictions'] = prediction_fn(logits, scope='Predictions')
      return logits, end_points

In [0]:
def time_tensorflow_run(session, target, info_string):
    num_steps_burn_in = 10
    total_duration = 0.0
    total_duration_squared = 0.0

    for i in range(num_batches + num_steps_burn_in):
        start_time = time.time()
        _ = session.run(target)
        duration = time.time() - start_time

        if i >= num_steps_burn_in:
            if not i % 10:
                print('%s:step %d,duration = %.3f' % (datetime.now(), i - num_steps_burn_in, duration))

            total_duration += duration
            total_duration_squared += duration * duration
    mn = total_duration / num_batches
    vr = total_duration_squared / num_batches - mn * mn
    sd = math.sqrt(vr)
    print('%s:%s across %d steps,%.3f +/- %.3f sec / batch' %
          (datetime.now(), info_string, num_batches, mn, sd))

In [6]:
batch_size = 32
height, width = 299, 299
inputs = tf.random_uniform((batch_size, height, width, 3))
with slim.arg_scope(inception_v3_arg_scope()):
    logits, end_points = inception_v3(inputs, is_training=False)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
num_batches = 100
time_tensorflow_run(sess, logits, "Forward")

2018-05-04 15:53:10.335857:step 0,duration = 0.298
2018-05-04 15:53:13.312992:step 10,duration = 0.300
2018-05-04 15:53:16.291989:step 20,duration = 0.300
2018-05-04 15:53:19.279831:step 30,duration = 0.301
2018-05-04 15:53:22.275689:step 40,duration = 0.297
2018-05-04 15:53:25.269843:step 50,duration = 0.300
2018-05-04 15:53:28.252936:step 60,duration = 0.300
2018-05-04 15:53:31.245630:step 70,duration = 0.300
2018-05-04 15:53:34.239912:step 80,duration = 0.297
2018-05-04 15:53:37.245823:step 90,duration = 0.300
2018-05-04 15:53:39.941418:Forward across 100 steps,0.299 +/- 0.002 sec / batch
